# PINN applied to the N-body Problem

In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline

import imageio
import os

In [2]:
import numpy as np

def generate_nbody_data(n_bodies, masses):
    # Generar posiciones y velocidades aleatorias para los cuerpos
    positions = np.random.uniform(-1, 1, size=(n_bodies, 2))
    velocities = np.random.normal(scale=0.1, size=(n_bodies, 2))

    # Calcular la aceleración inicial para cada cuerpo
    G = 1
    acceleration = np.zeros((n_bodies, 2))
    for i in range(n_bodies):
        for j in range(n_bodies):
            if i != j:
                r_ij = positions[j] - positions[i]
                acceleration[i] += G * masses[j] * r_ij / np.linalg.norm(r_ij)**3

    return positions, velocities, acceleration


In [3]:
class NBodyDataset(Dataset):
    def __init__(self, n_bodies, masses, n_points=100):
        self.positions, self.velocities, self.accelerations = generate_nbody_data(n_bodies, masses)
        self.times = torch.linspace(0, 1, n_points)

    def __len__(self):
        return len(self.times)

    def __getitem__(self, idx):
        t = self.times[idx]
        x = self.positions + self.velocities * t.unsqueeze(1) + 0.5 * self.accelerations * t.unsqueeze(1)**2
        v = self.velocities + self.accelerations * t.unsqueeze(1)
        a = self.accelerations
        return x, v, a


In [4]:
# n_points = 1000
# n_bodies = 5
# m_min, m_max = 1.0, 10.0
# x_min, x_max = -1.0, 1.0
# v_max = 0.1
# dt = 0.01

# dataset = NBodyDataset(n_points, n_bodies, m_min, m_max, x_min, x_max, v_max, dt)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

n_bodies = 5
masses = torch.rand(n_bodies)
n_points = 1000

dataset = NBodyDataset(n_bodies, masses, n_points)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)


TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [ ]:
class NBodyPINN(torch.nn.Module):
    def __init__(self, n_bodies, n_hidden):
        super().__init__()
        self.n_bodies = n_bodies
        self.n_hidden = n_hidden
        self.fc1 = torch.nn.Linear(3 * n_bodies, n_hidden)
        self.fc2 = torch.nn.Linear(n_hidden, n_hidden)
        self.fc3 = torch.nn.Linear(n_hidden, n_hidden)
        self.fc4 = torch.nn.Linear(n_hidden, 3 * n_bodies)
    
    def forward(self, x):
        x = x.view(-1, 3 * self.n_bodies)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x.view(-1, self.n_bodies, 3)


In [ ]:
model = NBodyPINN(n_bodies, n_hidden=50)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 100
train_losses = []
valid_losses = []

for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    for i, (x, v) in enumerate(dataloader):
        optimizer.zero_grad()
        x.requires_grad = True
        v.requires_grad = True
        pred_v = model(x)
        pred_a = torch.autograd.grad(pred_v.sum(), x, create_graph=True)[0]
        true_a = -(1.0 / torch.norm(x.unsqueeze(1) - x.unsqueeze(0), dim=-1)**3) @ x
        loss = criterion(pred_a, true_a) + criterion(pred_v, v)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    train_loss /= len(dataloader)
    train_losses.append(train_loss)
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for i, (x, v) in enumerate(dataloader):
            x.requires_grad = True
            v.requires_grad = True
            pred_v = model(x)
            pred_a = torch.autograd.grad(pred_v.sum(), x, create_graph=True)[0]
            true_a = -(1.0 / torch.norm(x.unsqueeze(1) - x.unsqueeze(0), dim=-1)**3) @ x
            loss = criterion(pred_a, true_a) + criterion(pred_v, v)
            valid_loss += loss.item()
        valid_loss /= len(dataloader)
        valid_losses.append(valid_loss)
    print(f"Epoch {epoch + 1}/{n_epochs}, Train Loss: {train_loss:.6f}, Valid Loss: {valid_loss:.6f}")


c:\Users\Det-Pc\anaconda3\envs\pytorch-C-DL\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100, 100, 3])) that is different to the input size (torch.Size([100, 5, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (5) must match the size of tensor b (100) at non-singleton dimension 1